In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/75-pix/images.npy
/kaggle/input/75-pix/labels.npy
/kaggle/input/150-pix/images.npy
/kaggle/input/150-pix/labels.npy


In [3]:
import tensorflow as tf

In [4]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.keras.initializers import HeUniform, GlorotUniform
import pandas as pd
import keras
import matplotlib.pyplot as plt
import numpy as np

images = np.load("/kaggle/input/75-pix/images.npy")
labels = np.load("/kaggle/input/75-pix/labels.npy")



In [21]:
#make labels for hour and minutes
def preprocess_labels_com(labels, num_classes):
    
    hour_labels = labels[:,0]
    hour_labels = k.utils.to_categorical(hour_labels, num_classes)
    minute_labels = labels[:,1]/60
    print(minute_labels)
    return hour_labels, minute_labels




In [22]:

num_classes = len(np.unique(labels[:,0]))
X_train1, X_test,y_train1,  y_test = train_test_split(images, labels, test_size=0.10, random_state=42) 
X_train, X_valid,y_train,  y_valid = train_test_split(X_train1, y_train1, test_size=0.10, random_state=42) 


X_train = X_train.astype('float32')
X_train = X_train/255.0
X_valid = X_valid/255.0
 
img_rows = X_train.shape[1]
img_cols = X_train.shape[2]

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_valid = X_valid.reshape(X_valid.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_valid = X_valid.reshape(X_valid.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

    
y_train_hours, y_train_minutes = preprocess_labels_com(y_train, num_classes)

y_valid_hours, y_valid_minutes = preprocess_labels_com(y_valid, num_classes)
y_test_hours, y_test_minutes = preprocess_labels_com(y_test, num_classes)



#set the model for the training


[0.38333333 0.45       0.43333333 ... 0.81666667 0.26666667 0.71666667]
[0.78333333 0.61666667 0.26666667 ... 0.76666667 0.78333333 0.3       ]
[0.7        0.98333333 0.13333333 ... 0.06666667 0.13333333 0.45      ]


In [23]:
X_test = X_test.astype('float32')
X_test = X_test/255.0
X_test = X_test.reshape(X_test.shape[0],  img_rows, img_cols, 1)

In [24]:
print(y_valid_minutes)


[0.78333333 0.61666667 0.26666667 ... 0.76666667 0.78333333 0.3       ]


In [25]:
from tensorflow import keras as k

batch_size = 64

##### need to modify this for minutes
# def reg_loss_function(y_true, y_pred):
#     #Calculate the absolute error
#     abs_diff = tf.abs(y_true - y_pred)
#     wrapped_diff = tf.abs(12 - abs_diff)

#     below_min_penalty = tf.reduce_sum(tf.square(tf.maximum(0.0, 0 - y_pred)))
#     above_max_penalty = tf.reduce_sum(tf.square(tf.maximum(0.0, y_pred - 12)))

#     range_penalty = below_min_penalty + above_max_penalty

#     # Calculate the minimum error
#     min_error = tf.minimum(abs_diff, wrapped_diff)

#     # Return the mean squared of the minimum error
#     return tf.reduce_mean(tf.square(min_error) + 0.1 * range_penalty)

input_ = k.layers.Input(shape=X_train.shape[1:])

# convolutional layers
c1 = k.layers.Conv2D(32, kernel_size=(3, 3),
                    activation='relu',padding='same',
                    input_shape=input_shape)(input_)
m1 = k.layers.MaxPooling2D(pool_size=(2, 2))(c1)
b1 = k.layers.BatchNormalization()(m1)

#2nd layers
c2 = k.layers.Conv2D(64, kernel_size=(3, 3),
                    activation='relu',padding='same',
                    input_shape=input_shape)(b1)
d2=   k.layers.Dropout(0.10)(c2)
m2 = k.layers.MaxPooling2D(pool_size=(2, 2))(d2)
b2 = k.layers.BatchNormalization()(m2)

#3rd layers
c3 = k.layers.Conv2D(128, kernel_size=(3, 3),
                    activation='relu',padding='same',
                    input_shape=input_shape)(b2)
d3 = k.layers.Dropout(0.10)(c3)
m3 = k.layers.MaxPooling2D(pool_size=(2, 2))(d3)
b3 = k.layers.BatchNormalization()(m3)

                           
#4th layers
c4 = k.layers.Conv2D(256, kernel_size=(3, 3),
                    activation='relu',padding='same',
                    input_shape=input_shape)(b3)
d4 = k.layers.Dropout(0.10)(c4)
m4 = k.layers.MaxPooling2D(pool_size=(2, 2))(d4)
b4 = k.layers.BatchNormalization()(m4)
                           

#final layer begins
f = k.layers.Flatten()(b4)
                           
                           
#for hours final layers
dense_hour1 = k.layers.Dense(128, activation='relu')(f)
dense_dropout1 = k.layers.Dropout(0.10)(dense_hour1)
final_hour = k.layers.Dense(num_classes, activation='softmax')(dense_dropout1)


# for minutes final layers
minute_dense1 = k.layers.Dense(512, activation='relu')(f)

minute_dropout1 = k.layers.Dropout(0.5)(minute_dense1)
minute_dense2 = k.layers.Dense(128, activation='relu')(minute_dropout1)
minute_dropout2 =  k.layers.Dropout(0.3)(minute_dense2)
    
minute_dense3 = k.layers.Dense(64, activation='relu')(minute_dropout2)
minute_dropout3 =  k.layers.Dropout(0.1)(minute_dense3)

final_minute = k.layers.Dense(1, activation='linear')(minute_dropout3)
                          
model = k.Model(inputs=input_, outputs=[final_hour, final_minute])
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 75, 75, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 75, 75,    │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 37, 37,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 37, 37,    │        128 │ max_pooling2d[0]… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 37, 37,    │     18,496 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 37, 37,    │          0 │ conv2d_1[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 18, 18,    │          0 │ dropout[0][0]     │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 18, 18,    │        256 │ max_pooling2d_1[… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 18, 18,    │     73,856 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 18, 18,    │          0 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 9, 9, 128) │          0 │ dropout_1[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 128) │        512 │ max_pooling2d_2[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 9, 9, 256) │    295,168 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 9, 9, 256) │          0 │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 4, 4, 256) │          0 │ dropout_2[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 4, 256) │      1,024 │ max_pooling2d_3[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 4096)      │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 512)       │  2,097,664 │ flatten[0][0]   

 Total params: 3,087,373 (11.78 MB)

 Trainable params: 3,086,413 (11.77 MB)

 Non-trainable params: 960 (3.75 KB)

In [26]:
from tensorflow.keras.losses import categorical_crossentropy, mse



model.compile(loss=['categorical_crossentropy', 'mse'], 
              optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
              metrics=['accuracy', 'mae'])


checkpoint_cb = keras.callbacks.ModelCheckpoint(
        f"combined_model.keras", save_best_only=True
    )
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',  # metric to monitor
    patience=10,          # number of epochs to wait for improvement
    restore_best_weights=True  # restore the best weights after stopping
)
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6
)



history = model.fit( X_train, [y_train_hours, y_train_minutes], 
        
        epochs=200,
        verbose = 1,
        validation_data=(X_valid, [y_valid_hours, y_valid_minutes]),
        callbacks=[checkpoint_cb, early_stopping, reduce_lr])

Epoch 1/200


I0000 00:00:1730772047.418121   10706 service.cc:145] XLA service 0x7c446c003890 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730772047.418173   10706 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730772047.418178   10706 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 13/456 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - dense_1_accuracy: 0.0897 - dense_5_mae: 0.7878 - loss: 4.0481  

I0000 00:00:1730772066.518198   10706 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


456/456 ━━━━━━━━━━━━━━━━━━━━ 51s 60ms/step - dense_1_accuracy: 0.0929 - dense_5_mae: 0.4562 - loss: 2.9522 - val_dense_1_accuracy: 0.0815 - val_dense_5_mae: 0.2789 - val_loss: 2.7505 - learning_rate: 1.0000e-04
Epoch 2/200
456/456 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - dense_1_accuracy: 0.1456 - dense_5_mae: 0.2826 - loss: 2.4847 - val_dense_1_accuracy: 0.1531 - val_dense_5_mae: 0.2946 - val_loss: 2.4362 - learning_rate: 1.0000e-04
Epoch 3/200
456/456 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - dense_1_accuracy: 0.1977 - dense_5_mae: 0.2597 - loss: 2.2762 - val_dense_1_accuracy: 0.2074 - val_dense_5_mae: 0.2965 - val_loss: 2.2353 - learning_rate: 1.0000e-04
Epoch 4/200
456/456 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - dense_1_accuracy: 0.2575 - dense_5_mae: 0.2479 - loss: 2.0417 - val_dense_1_accuracy: 0.2901 - val_dense_5_mae: 0.2774 - val_loss: 1.9853 - learning_rate: 1.0000e-04
Epoch 5/200
456/456 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - dense_1_accuracy: 0.3330 - dense_5_mae: 0.2356 - loss: 1.8000 - val

In [34]:
model.save(f'final_model_combined_{X_train.shape[1]}.keras')
data_history = pd.DataFrame(history.history)

In [36]:
data_history.to_csv('final_combined_report1.csv')

In [29]:
y_pred_main, y_pred_aux = model.predict(X_test)

57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step


In [32]:
print(y_pred_main[0].argmax(axis=-1), y_pred_aux[0][0]*60)
print(y_test[0])

1 28.46132755279541
[ 1 42]


In [33]:
total_loss, hour_loss, minute_loss = model.evaluate(
        X_test, [y_test_hours, y_test_minutes])


57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - dense_1_accuracy: 0.9565 - dense_5_mae: 0.2262 - loss: 0.2074
